In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
Transactions=pd.read_excel('2022 Case challenge_ DA update.xlsx',sheet_name='Data. Transactions')

In [20]:
Transactions=Transactions[['User_id','Order_id','DATE','GMV','Service Group','MerchantID']]


In [21]:
Transactions['User_id']=Transactions['User_id'].astype(str)
Transactions['Order_id']=Transactions['Order_id'].astype(str)
Transactions['MerchantID']=Transactions['MerchantID'].astype(str)

In [23]:
Transactions.head()

,User_id,Order_id,DATE,GMV,Service Group,MerchantID
0,61386143,8733622706,2021-01-01,100000,marketplace,37
1,48453125,8726857991,2021-01-01,5000,supermarket,9
2,49921027,8737326894,2021-01-01,106600,supermarket,9
3,46022523,8732579078,2021-01-01,270000,supermarket,9
4,44014594,8725567343,2021-01-01,68000,cvs,8


In [9]:
Loyalty_Points=pd.read_excel('2022 Case challenge_ DA update.xlsx',sheet_name='Data. Loyalty Points')

In [13]:
Loyalty_Points

,Service Group,Point Mechanism,Maximum Point Per Trans
0,supermarket,1 points/1000VND GMV,500 points
1,data,10 points/1000VND GMV,1000 points
2,cvs,1 points/1000VND GMV,300 points
3,marketplace,1 points/1000VND GMV,500 points
4,Coffee chains and Milk tea,1 points/1000VND GMV,500 points
5,Offline Beverage,1 points/1000VND GMV,300 points


In [14]:
Loyalty_Benefits=pd.read_excel('2022 Case challenge_ DA update.xlsx',sheet_name='Data. Loyalty benefits')

In [15]:
Loyalty_Benefits

,Class ID,Group,%cashback
0,2,cvs,5
1,2,Offline Beverage,5
2,2,data,5
3,3,cvs,7
4,3,Offline Beverage,7
5,3,data,7
6,3,marketplace,2
7,3,supermarket,2
8,4,cvs,12
9,4,Offline Beverage,12


# Part 1

1